In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome('C:\\Users\82104\data-analysis\chromedriver.exe')

In [11]:
driver.get("http://www.youtube.com")
driver.implicitly_wait(time_to_wait=2)
driver.maximize_window()

In [12]:
# 올해안의 동영상 조회수 순으로 나열
def youtube_searching(word):
    url = 'https://www.youtube.com/results?search_query=' + word+'&sp=CAMSBAgFEAE%253D'
    return url

In [13]:
word = "브이로그"
url = youtube_searching(word)
driver.get(url)

In [14]:
import time
body = driver.find_element_by_tag_name('body')
num_of_pagedowns = 5
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [15]:
# 웹페이지의 html 다운로드
html = driver.page_source

In [16]:
from bs4 import BeautifulSoup
page = BeautifulSoup(html,'html.parser')

In [17]:
# 제목
import unicodedata

title = page.find('a',attrs = {"id":"video-title"}).get_text().strip()
title =  unicodedata.normalize('NFC',title)
title

'(Sub)🌈알록달록이들🌈 / 음료 주제에 이렇게 이쁘다니..🤦🏻\u200d♀️ / cafe vlog / 카페 브이로그 / 더리터 / asmr'

In [18]:
#채널명
channel = page.find('a',attrs = {"class":"yt-simple-endpoint style-scope yt-formatted-string"}).get_text()
channel

'나징NAJING'

In [19]:
# 조회수
views = page.find('span',attrs = {"class":"style-scope ytd-video-meta-block"}).get_text()
views

'조회수 1541만회'

In [20]:
# # 영상길이
lengths = page.find('span',attrs = {"class":"style-scope ytd-thumbnail-overlay-time-status-renderer"}).get_text().strip()
lengths

'11:06'

In [21]:
# all_videos
videos = page.select('#contents > ytd-video-renderer')
videos

[<ytd-video-renderer class="style-scope ytd-item-section-renderer" inline-title-icon="" lockup="true" prominent-thumb-style="DEFAULT" use-prominent-thumbs=""><!--css-build:shady--><div class="style-scope ytd-video-renderer" id="dismissible">
 <ytd-thumbnail class="style-scope ytd-video-renderer" use-hovered-property=""><!--css-build:shady--><a aria-hidden="true" class="yt-simple-endpoint inline-block style-scope ytd-thumbnail" href="/watch?v=KDZ6CEtZ8KI" id="thumbnail" rel="null" tabindex="-1">
 <yt-image alt="" class="style-scope ytd-thumbnail" disable-upgrade="" ftl-eligible="" hidden="">
 </yt-image>
 <yt-img-shadow class="style-scope ytd-thumbnail no-transition" ftl-eligible="" loaded="" style="background-color: transparent;"><!--css-build:shady--><img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/KDZ6CEtZ8KI/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLDGbXEcA6InbUwKKUMh2BU1Evpe9Q" width="360"/></yt-img-shadow>
 <d

In [22]:
# 데이터 가져오기
def get_content(driver,video):
    # 1. 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    # 2. 제목 가져오기
    title = video.find('a',attrs = {"id":"video-title"}).get_text().strip()
    #채널명
    channel = video.find('a',attrs = {"class":"yt-simple-endpoint style-scope yt-formatted-string"}).get_text()
    # 조회수
    views = video.find('span',attrs = {"class":"style-scope ytd-video-meta-block"}).get_text()
    # 영상길이
    lengths = video.find('span',attrs = {"class":"style-scope ytd-thumbnail-overlay-time-status-renderer"}).get_text().strip()
    # 수집한 정보 저장하기
    
    data = [title,channel,lengths,views]
    return data


In [23]:
from tqdm import tqdm
# all
results = []
for video in tqdm(videos):
    data = get_content(driver,video)
    results.append(data)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.35s/it]


In [24]:
# 날짜 리스트
dates=[]
for i in range(len(videos)):
    date = page.select('#metadata-line > span')[2*i+1].text.strip()
    dates.append(date)

In [25]:
import pandas as pd 
result_df = pd.DataFrame(results)
dates_df = pd.DataFrame(dates)
results2 = pd.concat([result_df,dates_df],axis =1)
results2.columns = ['title','channelName','length','views','date']
results2

,title,channelName,length,views,date
0,(Sub)🌈알록달록이들🌈 / 음료 주제에 이렇게 이쁘다니..🤦🏻‍♀️ / cafe ...,나징NAJING,11:06,조회수 1541만회,10개월 전
1,여수 풀빌라 4k 브이로그. 어떤 수영복이 더 좋아? 40만 구독 기념.,Velvet Tube 벨벳튜브,19:50,조회수 583만회,10개월 전
2,LA vlog,Jennierubyjane Official,8:22,조회수 573만회,2주 전
3,캠핑 브이로그 . 하늘에 구멍이 뚫린듯 거센 빗소리 . 폭우 우중캠핑,류캠프 RYUCAMP,14:40,조회수 535만회,8개월 전
4,(재업 요청이 많았던) 라면먹는 꽃빵 브이로그 / Daily VLOG Last wi...,꽃빵 TV GGotBBang TV,10:01,조회수 395만회,9개월 전
5,캠핑 브이로그 . 모든걸 삼켜버릴 압도적인 빗소리 . 폭우 우중캠핑,류캠프 RYUCAMP,14:30,조회수 362만회,5개월 전
6,브이로그 찍다가 복권 당첨돼서 바로 퇴사 선언한 신입사원,빠더너스 BDNS,SHORTS,조회수 351만회,1개월 전
7,[Vlog] 같이 등산 할래요? 힐링 등산 브이로그,Velvet Tube 벨벳튜브,19:54,조회수 339만회,11개월 전
8,감성 브이로그의 현실... #shorts,SI LIVE,SHORTS,조회수 327만회,1개월 전
9,우리의 브이로그,새벽을여는건,8:52,조회수 294만회,7개월 전


In [ ]:
results2.to_excel('./files/'+word+'_crawling.xlsx')

In [26]:
# 조회수 높은 영상이 많은 채널분류 
from collections import Counter
channelNames = []
for i in results2['channelName']:
    channelNames.append(i)
channelNames_count = Counter(channelNames)
most_channel = channelNames_count.most_common(20)
most_channel_df = pd.DataFrame(most_channel)
most_channel_df
most_channel

[('나징NAJING', 3),
 ('살빼조DietJo', 3),
 ('Velvet Tube 벨벳튜브', 2),
 ('류캠프 RYUCAMP', 2),
 ('yoovlog', 2),
 ('Jennierubyjane Official', 1),
 ('꽃빵 TV GGotBBang TV', 1),
 ('빠더너스 BDNS', 1),
 ('SI LIVE', 1),
 ('새벽을여는건', 1),
 ('NCT DREAM', 1),
 ('강유미 yumi kang좋아서 하는 채널', 1),
 ('은재', 1)]

In [28]:
# 채널명 리스트화
channel_names =[]
for i in range(0,12):
    channel_names.append(most_channel[i][0])
channel_names

['나징NAJING',
 '살빼조DietJo',
 'Velvet Tube 벨벳튜브',
 '류캠프 RYUCAMP',
 'yoovlog',
 'Jennierubyjane Official',
 '꽃빵 TV GGotBBang TV',
 '빠더너스 BDNS',
 'SI LIVE',
 '새벽을여는건',
 'NCT DREAM',
 '강유미 yumi kang좋아서 하는 채널']

In [29]:
# 채널명 검색
def youtuber_searching(유튜버):
    url = 'https://www.youtube.com/results?search_query=' + 유튜버
    driver.get(url)
    time.sleep(2)

In [30]:
# 구독자 수 
html = driver.page_source
page = BeautifulSoup(html,'html.parser')
num_subscribers = page.find('span',attrs = {"id":"subscribers"}).get_text().strip().split(' ')[1]

AttributeError: 'NoneType' object has no attribute 'get_text'

In [31]:
# channel_names 전체 구독자수 list화
bychannel_subscriber_num = []
for i in tqdm(channel_names):
    youtuber_searching(i)
    html = driver.page_source
    page = BeautifulSoup(html,'html.parser')
    try:
        num_subscribers = page.find('span',attrs = {"id":"subscribers"}).get_text().strip().split(' ')[1]
    except:
        num_subscribers = '표시안함'
    bychannel_subscriber_num.append(num_subscribers)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:18<00:00,  6.50s/it]


In [32]:
bychannel_subscriber_num_df = pd.DataFrame(bychannel_subscriber_num)
most_channel_df= pd.concat([most_channel_df,bychannel_subscriber_num_df],axis =1)
most_channel_df.columns = ['channel','frequency','subscribers']
most_channel_df

,channel,frequency,subscribers
0,나징NAJING,3,113만명
1,살빼조DietJo,3,37.1만명
2,Velvet Tube 벨벳튜브,2,67.3만명
3,류캠프 RYUCAMP,2,14.2만명
4,yoovlog,2,36.5만명
5,Jennierubyjane Official,1,808만명
6,꽃빵 TV GGotBBang TV,1,51.2만명
7,빠더너스 BDNS,1,55만명
8,SI LIVE,1,3.19만명
9,새벽을여는건,1,18.7만명


In [33]:
titles = []
for i in results2['title']:
    titles.append(i)
str(titles)

"['(Sub)🌈알록달록이들🌈 / 음료 주제에 이렇게 이쁘다니..🤦🏻\\u200d♀️ / cafe vlog / 카페 브이로그 / 더리터 / asmr', '여수 풀빌라 4k 브이로그. 어떤 수영복이 더 좋아? 40만 구독 기념.', 'LA vlog', '캠핑 브이로그 . 하늘에 구멍이 뚫린듯 거센 빗소리 . 폭우 우중캠핑', '(재업 요청이 많았던) 라면먹는 꽃빵 브이로그 / Daily VLOG Last winter デイリーVLOG去年の冬 Harian VLOG Musim dingin lalu', '캠핑 브이로그 . 모든걸 삼켜버릴 압도적인 빗소리 . 폭우 우중캠핑', '브이로그 찍다가 복권 당첨돼서 바로 퇴사 선언한 신입사원', '[Vlog] 같이 등산 할래요? 힐링 등산 브이로그', '감성 브이로그의 현실... #shorts', '우리의 브이로그', '(Sub)🍑카페브이로그 30분 모아보기🍑 / 30 mins Cafe Vlog', 'Cafe vlog｜Cafe 7 DREAM을 찾아온 특별 손님🤭｜카페브이로그｜카페7드림｜#카페일상｜#카페브레이크타임｜#감성카페', '일진의 하루 브이로그', '[먹방브이로그#26] 찾았다 꿀조합😍 꼭 이렇게 드셔보세요ㅠㅠ🙏🏻타코야끼,불닭,신전로제떡볶이,싸이버거,치즈볼,떡강정,치즈케이크,크로플고피자,까르보나라,브라우니,티라미수찰떡꼬치', '[vlog] 피시방 알바 브이로그/벌써 여덟 개째네', '[먹방 브이로그#31] 죄송합니다 만칼로리 챌린지 말고 그냥 맛있게 먹을게요😖❤️ 슈프림양념치킨,도미노 포테이토피자,쿠키반민트반,불닭+짜파게티+타코야끼,태리로제떡볶이', '[Eng]출산특집#3 조리원 브이로그✨천국이라는 그곳...☁️아기 이름, 완모의 길', '[Eng]출산특집#1 출산 브이로그🤰🏻자연 분만, 일주일 진통 기다리며 먹부림🍛, 순산 기 받아가세요!!', '(Sub)❓🙉설마 새해 첫 날부터,,?🙉❓/ cafe vlog / 카페 브이로그 / 더리터 브이로그 / asmr / nobgm', '[먹방 브이로그#25] 어디서 터지는 소리 안

In [34]:
import re
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]' , '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
    return cleaned_text
titles2=[]
for i in titles:
    titles2.append(clean_text(i))
# only_BMP_pattern = re.compile("["
#         u"\U00010000-\U0010FFFF"  #BMP characters 이외
#                            "]+", flags=re.UNICODE)
# print(only_BMP_pattern.sub(r'', str(titles)))
titles2

['🌈알록달록이들🌈  음료 주제에 이렇게 이쁘다니🤦🏻\u200d♀️     카페 브이로그  더리터  ',
 '여수 풀빌라 4 브이로그 어떤 수영복이 더 좋아 40만 구독 기념',
 ' ',
 '캠핑 브이로그  하늘에 구멍이 뚫린듯 거센 빗소리  폭우 우중캠핑',
 '재업 요청이 많았던 라면먹는 꽃빵 브이로그      デイリー去年の冬     ',
 '캠핑 브이로그  모든걸 삼켜버릴 압도적인 빗소리  폭우 우중캠핑',
 '브이로그 찍다가 복권 당첨돼서 바로 퇴사 선언한 신입사원',
 ' 같이 등산 할래요 힐링 등산 브이로그',
 '감성 브이로그의 현실 ',
 '우리의 브이로그',
 '🍑카페브이로그 30분 모아보기🍑  30   ',
 ' ｜ 7 을 찾아온 특별 손님🤭｜카페브이로그｜카페7드림｜카페일상｜카페브레이크타임｜감성카페',
 '일진의 하루 브이로그',
 '먹방브이로그26 찾았다 꿀조합😍 꼭 이렇게 드셔보세요ㅠㅠ🙏🏻타코야끼불닭신전로제떡볶이싸이버거치즈볼떡강정치즈케이크크로플고피자까르보나라브라우니티라미수찰떡꼬치',
 ' 피시방 알바 브이로그벌써 여덟 개째네',
 '먹방 브이로그31 죄송합니다 만칼로리 챌린지 말고 그냥 맛있게 먹을게요😖❤️ 슈프림양념치킨도미노 포테이토피자쿠키반민트반불닭짜파게티타코야끼태리로제떡볶이',
 '출산특집3 조리원 브이로그✨천국이라는 그곳☁️아기 이름 완모의 길',
 '출산특집1 출산 브이로그🤰🏻자연 분만 일주일 진통 기다리며 먹부림🍛 순산 기 받아가세요',
 '❓🙉설마 새해 첫 날부터🙉❓    카페 브이로그  더리터 브이로그    ',
 '먹방 브이로그25 어디서 터지는 소리 안나요 제 입 터지는 소리👄💥다이어트추천템 로제엽떡포테킹짜파게티크로플배스킨라빈스그림의빵프로틴초코볼베노프']